Helpers: 
* https://stackoverflow.com/questions/32173053/copy-columns-from-workbook-paste-in-second-sheet-of-second-workbook-openpyxl
* https://automatetheboringstuff.com/chapter12/

In [1]:
import sys, os, openpyxl

In [2]:
#Get the filename
dataDir = '../../Data'
xlFN = dataDir + os.sep + 'LA_SWUDS_BalanceSheet.xlsx'
outFN = dataDir + os.sep + 'LA_SWUDS_BalanceSheet1.xlsx'

In [10]:
#Open the workbook
wb = openpyxl.load_workbook(xlFN)

In [11]:
#Get the sheet
ws = wb.get_sheet_by_name('2000')

In [12]:
#Copy column B to C
for src, dst in zip(ws['B:B'], ws['D:D']):
    dst.value = src.value
    #if src.has_style:
    dst.style = src.style
    #dst.fill = src.fill
    #dst.number_format = src.number_format
    #dst.alignment = src.alignment

In [14]:
#Save
wb.save(outFN)